# Using compression

> Objectives:
> * How to compress chunked datasets
> * Learn how to fine-tune the HDF5 compression pipeline to suit your needs
> * How to use pandas for reading CSV files

## Load movielens datasets

In [1]:
import os
import numpy as np
import pandas as pd
import tables

In [2]:
# Import CSV files via pandas
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat.gz')

# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols)

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [3]:
movies.ftypes

movie_id     int64:dense
title       object:dense
genres      object:dense
dtype: object

In [4]:
ratings.ftypes

user_id           int64:dense
movie_id          int64:dense
rating            int64:dense
unix_timestamp    int64:dense
dtype: object

## Storing in HDF5/PyTables in compressed form

In [5]:
import os
import shutil
data_dir = "compression"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

In [6]:
def to_hdf5(ratings, movies, filters):
    
    class Ratings(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        movie_id = tables.Int32Col(pos=1)
        rating = tables.Int8Col(pos=2)
        unix_timestamp = tables.Int64Col(pos=3)
    
    class Movies(tables.IsDescription):
        movie_id = tables.Int32Col(pos=0)
        title = tables.StringCol(100, pos=1)
        genres = tables.StringCol(50, pos=2)
    
    def get_filename(filters):
        if filters.complevel != 0:
            complib = filters.complib if ":" not in filters.complib else filters.complib.replace(":", "-")
            shuffle = "shuffle" if filters.shuffle else "noshuffle"
            filename = "%s/%s-%d-%s.h5" % (data_dir, complib, filters.complevel, shuffle)
        else:
            filename = "%s/no-compressed.h5" % (data_dir,)
        return filename

    filename = get_filename(filters)
    print("Creating file:", filename)
    with tables.open_file(filename, "w") as f:
        table_ratings = f.create_table(f.root, "ratings", Ratings, filters=filters, expectedrows=len(ratings))
        table_ratings.append([ratings[col].values for col in ratings.ftypes.keys()])
        table_movies = f.create_table(f.root, "movies", Movies, filters=filters, expectedrows=len(movies))
        table_movies.append([movies[col].values for col in movies.ftypes.keys()])
    return filename

In [7]:
%%time
filters = tables.Filters(complevel=5)
h5file = to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
CPU times: user 389 ms, sys: 26.7 ms, total: 416 ms
Wall time: 455 ms


In [8]:
!ptdump -v {h5file}

/ (RootGroup) ''
/movies (Table(3883,), shuffle, zlib(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)
/ratings (Table(1000209,), shuffle, zlib(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)


### Exercise 1

PyTables comes with out-of-box support for a series of codecs.  Do a quick comparison between "zlib", "bzip2", and "blosc" for compression levels of 1 (fastest), 5 and 9 (slowest).  Which one compresses best?  Which one compresses faster?

Also, Blosc being a meta-compressor, it has support for different codecs internally that can be selected from PyTables in the "blosc:`codec`" form.  Do another comparison between internal Blosc codecs, namely, "blosc:blosclz" (the default), "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib" and "blosc:zstd".

Finally, avoid any compression totally (`complevel=0`).  How fast it is compared with existing codecs?

In [9]:
for complib in ("zlib", "bzip2", "blosc"):
    filters = tables.Filters(complevel=5, complib=complib)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
CPU times: user 373 ms, sys: 21.7 ms, total: 395 ms
Wall time: 396 ms
Creating file: compression/bzip2-5-shuffle.h5
CPU times: user 2.37 s, sys: 18.7 ms, total: 2.39 s
Wall time: 2.4 s
Creating file: compression/blosc-5-shuffle.h5
CPU times: user 68.7 ms, sys: 12.7 ms, total: 81.4 ms
Wall time: 82.2 ms


In [10]:
%ls -lh {data_dir}

total 27152
-rw-r--r--  1 faltet  staff   5.0M May 20 13:07 blosc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.1M May 20 13:07 bzip2-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.2M May 20 13:07 zlib-5-shuffle.h5


In [11]:
for complib in ("blosc:blosclz", "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib", "blosc:zstd"):
    filters = tables.Filters(complevel=5, complib=complib)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/blosc-blosclz-5-shuffle.h5
CPU times: user 88.4 ms, sys: 24.1 ms, total: 112 ms
Wall time: 116 ms
Creating file: compression/blosc-lz4-5-shuffle.h5
CPU times: user 37.7 ms, sys: 12.7 ms, total: 50.5 ms
Wall time: 51.9 ms
Creating file: compression/blosc-lz4hc-5-shuffle.h5
CPU times: user 340 ms, sys: 14.1 ms, total: 354 ms
Wall time: 357 ms
Creating file: compression/blosc-snappy-5-shuffle.h5
CPU times: user 39.1 ms, sys: 10.9 ms, total: 50 ms
Wall time: 51.2 ms
Creating file: compression/blosc-zlib-5-shuffle.h5
CPU times: user 306 ms, sys: 15.4 ms, total: 321 ms
Wall time: 327 ms
Creating file: compression/blosc-zstd-5-shuffle.h5
CPU times: user 179 ms, sys: 11.8 ms, total: 191 ms
Wall time: 192 ms


In [12]:
# Finally, the uncompressed case
filters = tables.Filters(complevel=0)
%time to_hdf5(ratings, movies, filters)

Creating file: compression/no-compressed.h5
CPU times: user 30.6 ms, sys: 30.2 ms, total: 60.7 ms
Wall time: 69 ms


'compression/no-compressed.h5'

In [13]:
%ls -lh {data_dir}

total 121712
-rw-r--r--  1 faltet  staff   5.0M May 20 13:07 blosc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.0M May 20 13:07 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 20 13:07 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.8M May 20 13:07 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 20 13:07 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 20 13:07 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 20 13:07 blosc-zstd-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.1M May 20 13:07 bzip2-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 20 13:07 no-compressed.h5
-rw-r--r--  1 faltet  staff   4.2M May 20 13:07 zlib-5-shuffle.h5


## Reading compressed datasets

In [14]:
files = list(os.walk(data_dir))[0][2]

In [15]:
files

['blosc-5-shuffle.h5',
 'blosc-blosclz-5-shuffle.h5',
 'blosc-lz4-5-shuffle.h5',
 'blosc-lz4hc-5-shuffle.h5',
 'blosc-snappy-5-shuffle.h5',
 'blosc-zlib-5-shuffle.h5',
 'blosc-zstd-5-shuffle.h5',
 'bzip2-5-shuffle.h5',
 'no-compressed.h5',
 'zlib-5-shuffle.h5']

In [16]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5
CPU times: user 42.1 ms, sys: 13.1 ms, total: 55.2 ms
Wall time: 54.1 ms
Reading file: blosc-blosclz-5-shuffle.h5
CPU times: user 41.1 ms, sys: 11.9 ms, total: 52.9 ms
Wall time: 59.9 ms
Reading file: blosc-lz4-5-shuffle.h5
CPU times: user 23.2 ms, sys: 7.52 ms, total: 30.7 ms
Wall time: 31.5 ms
Reading file: blosc-lz4hc-5-shuffle.h5
CPU times: user 25.7 ms, sys: 8.77 ms, total: 34.4 ms
Wall time: 38.1 ms
Reading file: blosc-snappy-5-shuffle.h5
CPU times: user 28.7 ms, sys: 7.67 ms, total: 36.3 ms
Wall time: 38.9 ms
Reading file: blosc-zlib-5-shuffle.h5
CPU times: user 84.3 ms, sys: 8.91 ms, total: 93.2 ms
Wall time: 98 ms
Reading file: blosc-zstd-5-shuffle.h5
CPU times: user 41.7 ms, sys: 6.29 ms, total: 48 ms
Wall time: 48.6 ms
Reading file: bzip2-5-shuffle.h5
CPU times: user 683 ms, sys: 11.6 ms, total: 694 ms
Wall time: 700 ms
Reading file: no-compressed.h5
CPU times: user 7.35 ms, sys: 9.98 ms, total: 17.3 ms
Wall time: 17.5 ms
Reading file: zlib-5

### Exercise 2

Which codec and compression level can read the fastest?  How it does compare with reading an uncompressed dataset?

### Exercise 3

Blosc can use multithreading for compressing/decompressing, although it is disabled by default.  You can enable a multithreaded Blosc in a series of ways, but perhaps the easiest is to set the "BLOSC_NTHREADS" environment variable to the desired number of threads (typically the available number of cores in your computer).

Execute the cell below and re-do the reading benchmarks and look at how the reading speed vary.  Pay special attention to the difference between the CPU times and wall times.

In [17]:
os.environ["BLOSC_NTHREADS"] = "4"  # set to any other number you prefer

In [18]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5
CPU times: user 37.8 ms, sys: 10.4 ms, total: 48.3 ms
Wall time: 50.6 ms
Reading file: blosc-blosclz-5-shuffle.h5
CPU times: user 41.3 ms, sys: 17.2 ms, total: 58.5 ms
Wall time: 63.3 ms
Reading file: blosc-lz4-5-shuffle.h5
CPU times: user 28.3 ms, sys: 19.3 ms, total: 47.6 ms
Wall time: 34.8 ms
Reading file: blosc-lz4hc-5-shuffle.h5
CPU times: user 25.4 ms, sys: 17.4 ms, total: 42.8 ms
Wall time: 32 ms
Reading file: blosc-snappy-5-shuffle.h5
CPU times: user 34.1 ms, sys: 18.6 ms, total: 52.7 ms
Wall time: 39.2 ms
Reading file: blosc-zlib-5-shuffle.h5
CPU times: user 81.5 ms, sys: 17.5 ms, total: 99 ms
Wall time: 59.6 ms
Reading file: blosc-zstd-5-shuffle.h5
CPU times: user 50.8 ms, sys: 18.2 ms, total: 68.9 ms
Wall time: 44.7 ms
Reading file: bzip2-5-shuffle.h5
CPU times: user 659 ms, sys: 10.8 ms, total: 670 ms
Wall time: 676 ms
Reading file: no-compressed.h5
CPU times: user 6.77 ms, sys: 9.6 ms, total: 16.4 ms
Wall time: 16.1 ms
Reading file: zlib-5-